# Lesson 2 Exercise 1 Solution: Creating Normalized Tables


<strong>
In this exercise we are going to walk through the basics of modeling data in normalized form. We will create tables in PostgreSQL, insert rows of data, and do simple JOIN SQL queries to show how these mutliple tables can work together.
</strong>

<strong>import libraries</strong>

In [18]:
import psycopg2 as pg

**Create a connection to the database, get a cursor, and set autocommit to true**

In [19]:
try: 
    conn = pg.connect("host=127.0.0.1 dbname=udacity user=postgres password=admin")
    print("connect")
except pg.Error as e: 
    print("Error: Could not make connection to the postgres database")
    print(e)
try: 
    cur = conn.cursor()
except pg.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)
conn.set_session(autocommit=True)

connect



**Let's imagine we have a table called Music Store.**

    Table Name: music_store
    column 0: Transaction Id
    column 1: Customer Name
    column 2: Cashier Name
    column 3: Year 
    column 4: Albums Purchased

In [20]:
# create table.

try:
    cur.execute("CREATE TABLE IF NOT EXISTS music_store (transaction_id int NOT NULL unique,\
    customer_name varchar, cashier_name varchar, year int, album_purchased text[]);")
except pg.Error as e:
    print("Error: Could not create table")

In [4]:
# insert data into table

try:
    cur.execute("INSERT INTO music_store(transaction_id, customer_name, cashier_name, year, album_purchased)\
    VALUES(%s, %s, %s, %s, %s)\
    ON CONFLICT(transaction_id)\
    DO NOTHING;", (1, "Ibrahim", "Ali", 2000, ["Hello", "World beauty"]))
except pg.Error as e:
    print("Error: could not add data to table")
    print(e)
    
try:
    cur.execute("INSERT INTO music_store(transaction_id, customer_name, cashier_name, year, album_purchased)\
    VALUES(%s, %s, %s, %s, %s)\
    ON CONFLICT(transaction_id)\
    DO NOTHING;", (2, "Jamal", "Ahmed", 2002, ["Yes", "Test"]))
except pg.Error as e:
    print("Error: could not add data to table")
    print(e)
    
try:
    cur.execute("INSERT INTO music_store(transaction_id, customer_name, cashier_name, year, album_purchased)\
    VALUES(%s, %s, %s, %s, %s)\
    ON CONFLICT(transaction_id)\
    DO NOTHING;", (3, "Ameen", "Ali", 2000, ["team", "FCBayren"]))
except pg.Error as e:
    print("Error: could not add data to table")
    print(e)

In [21]:
# print results
try:
    cur.execute('select * from music_store;')
except pg.Error as e:
    print("Error: Select *")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()


(1, 'Ibrahim', 'Ali', 2000, ['Hello', 'World beauty'])
(2, 'Jamal', 'Ahmed', 2002, ['Yes', 'Test'])
(3, 'Ameen', 'Ali', 2000, ['team', 'FCBayren'])


**Moving to 1st Normal Form (1NF)**

This data has not been normalized. To get this data into 1st normal form, we will need to remove any collections or list of data. We need to break up the list of songs into individual rows.

Table Name: music_store

    column 0: Transaction Id
    column 1: Customer Name
    column 2: Cashier Name
    column 3: Year 
    column 4: Albums Purchased

In [22]:
# create table.

try:
    cur.execute("CREATE TABLE IF NOT EXISTS music_store1 (transaction_id int,\
    customer_name varchar, cashier_name varchar, year int, album_purchased varchar);")
except pg.Error as e:
    print("Error: Could not create table")

In [7]:
# insert data into table

try:
    cur.execute("INSERT INTO music_store1 (transaction_id, customer_name, cashier_name, year, album_purchased)\
    VALUES(%s, %s, %s, %s, %s);", (1, "Ibrahim", "Ali", 2000, "World beauty"))
except pg.Error as e:
    print("Error: could not add data to table")
    print(e)
    
try:
    cur.execute("INSERT INTO music_store1 (transaction_id, customer_name, cashier_name, year, album_purchased)\
    VALUES(%s, %s, %s, %s, %s);", (1, "Ibrahim", "Ali", 2000, "Hello"))
except pg.Error as e:
    print("Error: could not add data to table")
    print(e)

try:
    cur.execute("INSERT INTO music_store1 (transaction_id, customer_name, cashier_name, year, album_purchased)\
    VALUES(%s, %s, %s, %s, %s);", (2, "Jamal", "Ahmed", 2002, "Yes"))
except pg.Error as e:
    print("Error: could not add data to table")
    print(e)

try:
    cur.execute("INSERT INTO music_store1 (transaction_id, customer_name, cashier_name, year, album_purchased)\
    VALUES(%s, %s, %s, %s, %s);", (2, "Jamal", "Ahmed", 2002, "Test"))
except pg.Error as e:
    print("Error: could not add data to table")
    print(e)

try:
    cur.execute("INSERT INTO music_store1 (transaction_id, customer_name, cashier_name, year, album_purchased)\
    VALUES(%s, %s, %s, %s, %s);", (3, "Ameen", "Ali", 2000, "team"))
except pg.Error as e:
    print("Error: could not add data to table")
    print(e)

try:
    cur.execute("INSERT INTO music_store1 (transaction_id, customer_name, cashier_name, year, album_purchased)\
    VALUES(%s, %s, %s, %s, %s);", (3, "Ameen", "Ali", 2000, "FCBayren"))
except pg.Error as e:
    print("Error: could not add data to table")
    print(e)

In [23]:
# print results
try:
    cur.execute('select * from music_store1 ;')
except pg.Error as e:
    print("Error: Select *")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Ibrahim', 'Ali', 2000, 'World beauty')
(1, 'Ibrahim', 'Ali', 2000, 'Hello')
(2, 'Jamal', 'Ahmed', 2002, 'Yes')
(2, 'Jamal', 'Ahmed', 2002, 'Test')
(3, 'Ameen', 'Ali', 2000, 'team')
(3, 'Ameen', 'Ali', 2000, 'FCBayren')


**Moving to 2nd Normal Form (2NF)**

We have moved our data to be in 1NF which is the first step in moving to 2nd Normal Form. Our table is not yet in 2nd Normal Form. While each of our records in our table is unique, our Primary key (transaction id) is not unique. We need to break this up into two tables, transactions and albums sold.

Table Name: transactions 

    column 0: Transaction ID
    column 1: Customer Name
    column 2: Cashier Name
    column 3: Year
    
Table Name: albums_sold

    column 0: Album Id
    column 1: Transaction Id
    column 3: Album Name

In [24]:
# create table.

try:
    cur.execute("CREATE TABLE IF NOT EXISTS transaction (transaction_id int,\
    customer_name varchar, cashier_name varchar, year int);")
except pg.Error as e:
    print("Error: Could not create table")


try:
    cur.execute("CREATE TABLE IF NOT EXISTS album_sold (album_id int, transaction_id int, album_name varchar);")
except pg.Error as e:
    print("Error: Could not create table")

In [10]:
# insert data into tables.

try: 
    cur.execute("INSERT INTO transaction (transaction_id, customer_name, cashier_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (1, "ibrahim", "Ali", 2000))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO transaction (transaction_id, customer_name, cashier_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (2, "Jamal", "Ahmed", 2000))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)
    
try: 
    cur.execute("INSERT INTO transaction (transaction_id, customer_name, cashier_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (3, "Ameen", "Ali", 2018))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)
    
try: 
    cur.execute("INSERT INTO album_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (1, 1, "Hello"))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO album_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (2, 1, "world beauty"))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)
    
try: 
    cur.execute("INSERT INTO album_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (3, 2, "yes"))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)
    
try: 
    cur.execute("INSERT INTO album_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (4, 2, "test"))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO album_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (5, 3, "team"))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO album_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (6, 3, "FCB"))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

In [25]:
# print results
try:
    cur.execute('select * from transaction;')
except pg.Error as e:
    print('Error: select *')
    print(e)

print('Transaction table')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    
try:
    cur.execute('select * from album_sold;')
except pg.Error as e:
    print('Error: select *')
    print(e)

print('\nalbum sold table')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

Transaction table
(1, 'ibrahim', 'Ali', 2000)
(2, 'Jamal', 'Ahmed', 2000)
(3, 'Ameen', 'Ali', 2018)

album sold table
(1, 1, 'Hello')
(2, 1, 'world beauty')
(3, 2, 'yes')
(4, 2, 'test')
(5, 3, 'team')
(6, 3, 'FCB')


In [12]:
# I complete the join on the transactions and album_sold tables

try: 
    cur.execute('select * from transaction t \
    join album_sold a on t.transaction_id = a.transaction_id')
except pg.Error as e:
    print("Error: join query")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'ibrahim', 'Ali', 2000, 1, 1, 'Hello')
(1, 'ibrahim', 'Ali', 2000, 2, 1, 'world beauty')
(2, 'Jamal', 'Ahmed', 2000, 3, 2, 'yes')
(2, 'Jamal', 'Ahmed', 2000, 4, 2, 'test')
(3, 'Ameen', 'Ali', 2018, 5, 3, 'team')
(3, 'Ameen', 'Ali', 2018, 6, 3, 'FCB')


**Moving to 3rd Normal Form (3NF)**

Let's check our table for any transitive dependencies. Transactions can remove Cashier Name to its own table, called Employees, which will leave us with 3 tables.  

Table Name: transactions2 

    column 0: transaction Id
    column 1: Customer Name
    column 2: Cashier Id
    column 3: Year

Table Name: albums_sold

    column 0: Album Id
    column 1: Transaction Id
    column 3: Album Name

Table Name: employees

    column 0: Employee Id
    column 1: Employee Name

In [13]:
# create table.

try:
    cur.execute("CREATE TABLE IF NOT EXISTS transaction2 (transaction_id int,\
    customer_name varchar, cashier_id int, year int);")
except pg.Error as e:
    print("Error: Could not create table")
    
try:
    cur.execute("CREATE TABLE IF NOT EXISTS employee (employee_id int,\
    employee_name varchar);")
except pg.Error as e:
    print("Error: Could not create table")


In [14]:
# insert data into tables

try: 
    cur.execute("INSERT INTO transaction2 (transaction_id, customer_name, cashier_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (1, "ibrahim", 1, 2000))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO transaction2 (transaction_id, customer_name, cashier_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (2, "ibrahim", 1, 2000))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO transaction2 (transaction_id, customer_name, cashier_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (3, "Jamal", 2, 2000))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO transaction2 (transaction_id, customer_name, cashier_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (4, "Jamal", 2, 2000))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO transaction2 (transaction_id, customer_name, cashier_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (5, "Ameen", 1, 2000))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO transaction2 (transaction_id, customer_name, cashier_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (6, "Ameen", 1, 2000))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO employee (employee_id, employee_name) \
                 VALUES (%s, %s)", \
                 (1, "Ali"))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

try: 
    cur.execute("INSERT INTO employee (employee_id, employee_name) \
                 VALUES (%s, %s)", \
                 (2, "Ahmed"))
except psycopg2.Error as e: 
    print("Error: could not add data to table")
    print (e)

In [15]:
# print results
try:
    cur.execute('select * from transaction2;')
except pg.Error as e:
    print('Error: select *')
    print(e)

print('Transaction table')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    
try:
    cur.execute('select * from album_sold;')
except pg.Error as e:
    print('Error: select *')
    print(e)

print('\nalbum sold table')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

try:
    cur.execute('select * from employee;')
except pg.Error as e:
    print('Error: select *')
    print(e)

print('\nemployee  table')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

Transaction table
(1, 'ibrahim', 1, 2000)
(2, 'ibrahim', 1, 2000)
(3, 'Jamal', 2, 2000)
(4, 'Jamal', 2, 2000)
(5, 'Ameen', 1, 2000)
(6, 'Ameen', 1, 2000)

album sold table
(1, 1, 'Hello')
(2, 1, 'world beauty')
(3, 2, 'yes')
(4, 2, 'test')
(5, 3, 'team')
(6, 3, 'FCB')

employee  table
(1, 'Ali')
(2, 'Ahmed')


In [17]:
# now JOIN on these 3 tables so we can get all the information we had in our first Table.
try: 
    cur.execute("select * from transaction2 join album_sold on \
                               transaction2.transaction_id = album_sold.transaction_id join \
                               employee on transaction2.cashier_id=employee.employee_id;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, 'ibrahim', 1, 2000, 1, 1, 'Hello', 1, 'Ali')
(1, 'ibrahim', 1, 2000, 2, 1, 'world beauty', 1, 'Ali')
(2, 'ibrahim', 1, 2000, 3, 2, 'yes', 1, 'Ali')
(2, 'ibrahim', 1, 2000, 4, 2, 'test', 1, 'Ali')
(3, 'Jamal', 2, 2000, 5, 3, 'team', 2, 'Ahmed')
(3, 'Jamal', 2, 2000, 6, 3, 'FCB', 2, 'Ahmed')


### So This is normalized database